Code adapted from NVIDIA OpenACC example: laplace2d-kernels.c

Copyright (c) 2012, NVIDIA CORPORATION. All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions
are met:
 * Redistributions of source code must retain the above copyright
   notice, this list of conditions and the following disclaimer.
 * Redistributions in binary form must reproduce the above copyright
   notice, this list of conditions and the following disclaimer in the
   documentation and/or other materials provided with the distribution.
 * Neither the name of NVIDIA CORPORATION nor the names of its
   contributors may be used to endorse or promote products derived
   from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS ''AS IS'' AND ANY
EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR
PURPOSE ARE DISCLAIMED.  IN NO EVENT SHALL THE COPYRIGHT OWNER OR
CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL,
EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO,
PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR
PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY
OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
(INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

In [1]:
%load_ext line_profiler

In [2]:
%load_ext snakeviz

In [3]:
import numpy as np
import time

In [4]:
import pickle

In [5]:
def np_laplace(n=4096, m=4096, iteration_max = 1000, debug=False):

    pi = np.pi
    tol = 1.0e-5
    error = 1.0

    A = np.zeros((n, m))

    j_range = np.arange(n)
    y0 = np.sin(pi * j_range / (n - 1))
    A[:, 0]  = y0
    A[:, -1] = y0 * np.exp(-pi)

    if debug: print('Jacobi relaxation Calculation: {} x {} mesh\n'.format(n, m))

    tic = time.time()
    iteration = 0

    Anew = np.copy(A)

    while error > tol and iteration < iteration_max:
        error = 0.0
        Anew[1:-1, 1:-1] = 0.25 * (A[1:-1, 2:] + A[1:-1, :-2] + A[2:, 1:-1] + A[:-2, 1:-1])      
        diff = np.abs(Anew[1:-1, 1:-1] - A[1:-1, 1:-1])
        error_new = np.max(diff)
        if error_new > error:
            error = error_new
        A[:] = Anew[:]

        if debug and not iteration % 100:
            print("{:5d}, {:0.6f}\n".format(iteration, error))

        iteration += 1

    toc = time.time() - tic

    if debug: print(" total: {} s\n".format(toc))
    
    return A

In [6]:
res = np_laplace(12, 12)

In [7]:
%lprun -f np_laplace np_laplace(512,512,debug=True) # do not use spaces, it confuses the profiler

Jacobi relaxation Calculation: 512 x 512 mesh

    0, 0.249999

  100, 0.002395

  200, 0.001202

  300, 0.000801

  400, 0.000601

  500, 0.000481

  600, 0.000400

  700, 0.000343

  800, 0.000300

  900, 0.000266

 total: 9.868297338485718 s



In [8]:
pow_max = 13
N = [2 ** val for val in range(2, pow_max)]

In [9]:
N

[4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]

In [10]:
np_res = {}
for n in N:
    res = %timeit -o -n 3 np_laplace(n, n)
    np_res[n] = res
pickle.dump(np_res, open('np_times.p', 'wb'))

3 loops, best of 3: 627 µs per loop
3 loops, best of 3: 3.03 ms per loop
3 loops, best of 3: 9.62 ms per loop
3 loops, best of 3: 41.5 ms per loop
3 loops, best of 3: 81.9 ms per loop
3 loops, best of 3: 297 ms per loop
3 loops, best of 3: 1.4 s per loop
3 loops, best of 3: 8.28 s per loop
3 loops, best of 3: 40 s per loop
3 loops, best of 3: 2min 51s per loop
3 loops, best of 3: 11min 41s per loop


In [11]:
np_res

{4: <TimeitResult : 3 loops, best of 3: 627 µs per loop>,
 8: <TimeitResult : 3 loops, best of 3: 3.03 ms per loop>,
 16: <TimeitResult : 3 loops, best of 3: 9.62 ms per loop>,
 32: <TimeitResult : 3 loops, best of 3: 41.5 ms per loop>,
 64: <TimeitResult : 3 loops, best of 3: 81.9 ms per loop>,
 128: <TimeitResult : 3 loops, best of 3: 297 ms per loop>,
 256: <TimeitResult : 3 loops, best of 3: 1.4 s per loop>,
 512: <TimeitResult : 3 loops, best of 3: 8.28 s per loop>,
 1024: <TimeitResult : 3 loops, best of 3: 40 s per loop>,
 2048: <TimeitResult : 3 loops, best of 3: 2min 51s per loop>,
 4096: <TimeitResult : 3 loops, best of 3: 11min 41s per loop>}

# Adding numba jit decorator

In [12]:
from numba import jit, njit

In [13]:
@jit(['float32[:,:], float32[:,:]',
      'float64[:,:], float64[:,:]'],
      nopython=True)
def update_a_jit(A, Anew):
    error = 0.0
    n, m = A.shape

    for j in range(1, n-1):
        for i in range(1, m-1):
            Anew[j, i] = 0.25 * (A[j, i+1] + A[j, i-1] + A[j-1, i] + A[j+1, i])
            error = max(error, abs(Anew[j, i] - A[j, i]))

    return Anew, error

In [14]:
# this did not make much difference
@jit(['float32[:,:], float32[:,:]',
      'float64[:,:], float64[:,:]'],
      nopython=True)
def a_to_anew_jit(A, Anew):
    error = 0.0
    n, m = A.shape
    
    for j in range(1, n-1):
        for i in range(1, m-1):
            A[j, i] = Anew[j, i]
    
    return A

In [15]:
def jit_laplace(n=4096, m=4096, iteration_max = 1000, debug=False):

    pi = np.pi
    tol = 1.0e-5
    error = 1.0

    A = np.zeros((n, m))

    j_range = np.arange(n)
    y0 = np.sin(pi * j_range / (n - 1))
    A[:, 0]  = y0
    A[:, -1] = y0 * np.exp(-pi)

    if debug: print('Jacobi relaxation Calculation: {} x {} mesh\n'.format(n, m))

    tic = time.time()
    iteration = 0

    Anew = np.copy(A)

    while error > tol and iteration < iteration_max:
        Anew, error = update_a_jit(A, Anew)
        # A[:] = Anew[:]
        A = a_to_anew_jit(A, Anew)  # this was essentially the same as the line above

        if debug and not iteration % 100:
            print("{:5d}, {:0.6f}\n".format(iteration, error))

        iteration += 1

    toc = time.time() - tic

    if debug: print(" total: {} s\n".format(toc))
    
    return A

In [16]:
res = jit_laplace(512, 512) # once to compile

In [17]:
%lprun -f jit_laplace jit_laplace(512,512,debug=True) # do not use spaces, it confuses the profiler

Jacobi relaxation Calculation: 512 x 512 mesh

    0, 0.249999

  100, 0.002395

  200, 0.001202

  300, 0.000801

  400, 0.000601

  500, 0.000481

  600, 0.000400

  700, 0.000343

  800, 0.000300

  900, 0.000266

 total: 1.4268712997436523 s



In [18]:
jit_res = {}
for n in N:
    res = %timeit -o -n 3 jit_laplace(n, n)
    jit_res[n] = res
pickle.dump(jit_res, open('jit_times.p', 'wb'))

3 loops, best of 3: 124 µs per loop
3 loops, best of 3: 272 µs per loop
3 loops, best of 3: 961 µs per loop
3 loops, best of 3: 4.93 ms per loop
3 loops, best of 3: 15.2 ms per loop
3 loops, best of 3: 63.4 ms per loop
3 loops, best of 3: 317 ms per loop
3 loops, best of 3: 1.36 s per loop
3 loops, best of 3: 8.95 s per loop
3 loops, best of 3: 37.4 s per loop
3 loops, best of 3: 3min 35s per loop


# Implementing `guvectorize`

In [19]:
from numba import guvectorize

In [20]:
from math import fabs

### CPU Vectorization

In [21]:
@guvectorize(['float32[:,:], float32[:,:]',
              'float64[:,:], float64[:,:]'],
             '(n,m)->(n,m)', nopython=True,
             target='cpu')
def update_a_serial0(A, Anew):
    n, m = A.shape
    for j in range(1, n-1):
        for i in range(1, m-1):
            Anew[j, i] = 0.25 * (A[j, i+1] + A[j, i-1] + A[j-1, i] + A[j+1, i])

In [22]:
@guvectorize(['float32[:,:], float32[:,:], float32[:]',
              'float64[:,:], float64[:,:], float64[:]'],
             '(n,m)->(n,m),()', nopython=True, target='cpu')
def update_a_serial(A, Anew, error):
    n, m = A.shape
    for j in range(1, n-1):
        for i in range(1, m-1):
            Anew[j, i] = 0.25 * (A[j, i+1] + A[j, i-1] + A[j-1, i] + A[j+1, i])

            diff = abs(Anew[j, i] - A[j, i])
            if diff > error[0]:
                error[0] = diff

In [23]:
A = np.zeros((4,4))
A[:, [0, -1]] = 1.0
Anew = np.zeros_like(A)
error = np.zeros(1)
Anew, error = update_a_serial(A, Anew, error)
np.max(np.abs(Anew[1:-1, 1:-1] - A[1:-1, 1:-1])), error[0]

(0.25, 0.25)

In [24]:
def cpu_laplace(n=4096, m=4096, iteration_max = 1000, debug=False):

    pi = np.pi
    tol = 1.0e-5
    error = np.ones(1)

    A = np.zeros((n, m))

    j_range = np.arange(n)
    y0 = np.sin(pi * j_range / (n - 1))
    A[:, 0]  = y0
    A[:, -1] = y0 * np.exp(-pi)

    if debug: print('Jacobi relaxation Calculation: {} x {} mesh\n'.format(n, m))

    tic = time.time()
    iteration = 0

    Anew = np.copy(A)
    while error[0] > tol and iteration < iteration_max:
        error[0] = 0.0
        Anew, error = update_a_serial(A, Anew, error)
        A[:] = Anew[:]

        if debug and not iteration % 100:
            print("{:5d}, {:0.6f}\n".format(iteration, error[0]))

        iteration += 1

    toc = time.time() - tic

    if debug: print(" total: {} s\n".format(toc))
    
    return A

In [25]:
res = cpu_laplace(512, 512)

Jacobi relaxation Calculation: 512 x 512 mesh

 total: 1.4511957168579102 s



In [26]:
%lprun -f cpu_laplace cpu_laplace(512,512,debug=True) # do not use spaces, it confuses the profiler

Jacobi relaxation Calculation: 512 x 512 mesh

    0, 0.249999

  100, 0.002395

  200, 0.001202

  300, 0.000801

  400, 0.000601

  500, 0.000481

  600, 0.000400

  700, 0.000343

  800, 0.000300

  900, 0.000266

 total: 1.6434290409088135 s



In [27]:
%timeit cpu_laplace(1024, 1024)

Jacobi relaxation Calculation: 1024 x 1024 mesh

 total: 9.51997995376587 s

Jacobi relaxation Calculation: 1024 x 1024 mesh

 total: 9.60679841041565 s

Jacobi relaxation Calculation: 1024 x 1024 mesh

 total: 9.44826602935791 s

Jacobi relaxation Calculation: 1024 x 1024 mesh

 total: 9.448802471160889 s

1 loop, best of 3: 9.45 s per loop


In [28]:
cpu_res = {}
for n in N:
    res = %timeit -o -n 3 cpu_laplace(n, n)
    cpu_res[n] = res
pickle.dump(cpu_res, open('cpu_times.p', 'wb'))

Jacobi relaxation Calculation: 4 x 4 mesh

 total: 0.00025177001953125 s

Jacobi relaxation Calculation: 4 x 4 mesh

 total: 0.0001475811004638672 s

Jacobi relaxation Calculation: 4 x 4 mesh

 total: 0.0001513957977294922 s

Jacobi relaxation Calculation: 4 x 4 mesh

 total: 0.00033354759216308594 s

Jacobi relaxation Calculation: 4 x 4 mesh

 total: 0.00015354156494140625 s

Jacobi relaxation Calculation: 4 x 4 mesh

 total: 0.00011444091796875 s

Jacobi relaxation Calculation: 4 x 4 mesh

 total: 0.00011348724365234375 s

Jacobi relaxation Calculation: 4 x 4 mesh

 total: 0.00011324882507324219 s

Jacobi relaxation Calculation: 4 x 4 mesh

 total: 0.00010752677917480469 s

3 loops, best of 3: 214 µs per loop
Jacobi relaxation Calculation: 8 x 8 mesh

 total: 0.0007557868957519531 s

Jacobi relaxation Calculation: 8 x 8 mesh

 total: 0.00032782554626464844 s

Jacobi relaxation Calculation: 8 x 8 mesh

 total: 0.0003230571746826172 s

Jacobi relaxation Calculation: 8 x 8 mesh

 total:

### parallel CPU vectorization

In [29]:
@guvectorize(['float32[:,:], float32[:,:], float32[:]',
              'float64[:,:], float64[:,:], float64[:]'],
             '(n,m)->(n,m),()', nopython=True, target='parallel')
def update_a_parallel(A, Anew, error):
    n, m = A.shape
    for j in range(1, n-1):
        for i in range(1, m-1):
            Anew[j, i] = 0.25 * (A[j, i+1] + A[j, i-1] + A[j-1, i] + A[j+1, i])

            diff = abs(Anew[j, i] - A[j, i])
            if diff > error[0]:
                error[0] = diff

In [30]:
def par_laplace(n=4096, m=4096, iteration_max = 1000, debug=False):

    pi = np.pi
    tol = 1.0e-5
    error = np.ones(1)

    A = np.zeros((n, m))

    j_range = np.arange(n)
    y0 = np.sin(pi * j_range / (n - 1))
    A[:, 0]  = y0
    A[:, -1] = y0 * np.exp(-pi)

    if debug: print('Jacobi relaxation Calculation: {} x {} mesh\n'.format(n, m))

    tic = time.time()
    iteration = 0

    Anew = np.copy(A)
    while error[0] > tol and iteration < iteration_max:
        error[0] = 0.0
        Anew, error = update_a_parallel(A, Anew, error)
        A[:] = Anew[:]

        if debug and not iteration % 100:
            print("{:5d}, {:0.6f}\n".format(iteration, error[0]))

        iteration += 1

    toc = time.time() - tic

    if debug: print(" total: {} s\n".format(toc))
    
    return A

In [31]:
res = par_laplace(512, 512, debug=False)

Jacobi relaxation Calculation: 512 x 512 mesh

 total: 2.518939971923828 s



In [32]:
%lprun -f par_laplace cpu_laplace(512,512,debug=True) # do not use spaces, it confuses the profiler

Jacobi relaxation Calculation: 512 x 512 mesh

    0, 0.249999

  100, 0.002395

  200, 0.001202

  300, 0.000801

  400, 0.000601

  500, 0.000481

  600, 0.000400

  700, 0.000343

  800, 0.000300

  900, 0.000266

 total: 1.5426313877105713 s



In [33]:
%timeit par_laplace(1024, 1024)

Jacobi relaxation Calculation: 1024 x 1024 mesh

 total: 12.103698015213013 s

Jacobi relaxation Calculation: 1024 x 1024 mesh

 total: 12.075923204421997 s

Jacobi relaxation Calculation: 1024 x 1024 mesh

 total: 12.05683422088623 s

Jacobi relaxation Calculation: 1024 x 1024 mesh

 total: 11.96055006980896 s

1 loop, best of 3: 12 s per loop


In [34]:
par_res = {}
for n in N:
    res = %timeit -o -n 3 par_laplace(n, n)
    par_res[n] = res
pickle.dump(par_res, open('par_times.p', 'wb'))

Jacobi relaxation Calculation: 4 x 4 mesh

 total: 0.0011873245239257812 s

Jacobi relaxation Calculation: 4 x 4 mesh

 total: 0.0009284019470214844 s

Jacobi relaxation Calculation: 4 x 4 mesh

 total: 0.0004956722259521484 s

Jacobi relaxation Calculation: 4 x 4 mesh

 total: 0.0005905628204345703 s

Jacobi relaxation Calculation: 4 x 4 mesh

 total: 0.0005083084106445312 s

Jacobi relaxation Calculation: 4 x 4 mesh

 total: 0.0006206035614013672 s

Jacobi relaxation Calculation: 4 x 4 mesh

 total: 0.0005619525909423828 s

Jacobi relaxation Calculation: 4 x 4 mesh

 total: 0.0007655620574951172 s

Jacobi relaxation Calculation: 4 x 4 mesh

 total: 0.0009443759918212891 s

3 loops, best of 3: 658 µs per loop
Jacobi relaxation Calculation: 8 x 8 mesh

 total: 0.004186391830444336 s

Jacobi relaxation Calculation: 8 x 8 mesh

 total: 0.006165266036987305 s

Jacobi relaxation Calculation: 8 x 8 mesh

 total: 0.00337982177734375 s

Jacobi relaxation Calculation: 8 x 8 mesh

 total: 0.002

### CUDA vectorization

In [35]:
@guvectorize(['float32[:,:], float32[:,:], float32[:]',
              'float64[:,:], float64[:,:], float64[:]'],
             '(n,m)->(n,m),()', nopython=True, target='cuda')
def update_a_cuda(A, Anew, error):
    n, m = A.shape
    for j in range(1, n-1):
        for i in range(1, m-1):
            Anew[j, i] = 0.25 * (A[j, i+1] + A[j, i-1] + A[j-1, i] + A[j+1, i])

            diff = abs(Anew[j, i] - A[j, i])
            if diff > error[0]:
                error[0] = diff

AssertionError: only support 1 output

In [ ]:
def cuda_laplace(n=4096, m=4096, iteration_max = 1000, debug=False):

    pi = np.pi
    tol = 1.0e-5
    error = np.ones(1)

    A = np.zeros((n, m))

    j_range = np.arange(n)
    y0 = np.sin(pi * j_range / (n - 1))
    A[:, 0]  = y0
    A[:, -1] = y0 * np.exp(-pi)

    if debug: print('Jacobi relaxation Calculation: {} x {} mesh\n'.format(n, m))

    tic = time.time()
    iteration = 0

    Anew = np.copy(A)
    while error[0] > tol and iteration < iteration_max:
        error[0] = 0.0
        Anew, error = update_a_cuda(A, Anew, error)
        A[:] = Anew[:]

        if debug and not iteration % 100:
            print("{:5d}, {:0.6f}\n".format(iteration, error[0]))

        iteration += 1

    toc = time.time() - tic

    if debug: print(" total: {} s\n".format(toc))
    
    return A

In [ ]:
res = cuda_laplace(512, 512, debug=False)

In [ ]:
%lprun -f cuda_laplace cuda_laplace(512,512,debug=True) # do not use spaces, it confuses the profiler

In [ ]:
cuda_res = {}
for n in N:
    res = %timeit -o -n 3 cuda_laplace(n, n)
    cuda_res[n] = res
pickle.dump(cuda_res, open('cuda_times.p', 'wb'))